In [ ]:
# third party
import numpy as np
import requests

# syft absolute
import syft as sy

# Verify Version

In [ ]:
PROJECT_NAME = "syft"
PYPI_JSON_URL = f"https://pypi.org/pypi/{PROJECT_NAME}/json"


def get_latest_pypi_version():
    response = requests.get(PYPI_JSON_URL)
    data = response.json()
    return data["info"]["version"]


latest_deployed_version = get_latest_pypi_version()

In [ ]:
# this notebook should only be used to run the latest deployed version of syft
# the notebooks after this (1a/1b and 2), will test migrating from that latest version
print(
    f"latest deployed version: {latest_deployed_version}, installed version: {sy.__version__}"
)
# assert (
#     latest_deployed_version == sy.__version__
# ), f"{latest_deployed_version} does not match installed version {sy.__version__}"

# Launch Server

In [ ]:
server = sy.orchestra.launch(
    name="test_upgradability",
    dev_mode=True,
    reset=True,
    n_consumers=2,
    create_producer=True,
    port="auto",
)

In [ ]:
client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
client.register(
    email="ds@openmined.org", name="John Doe", password="pw", password_verify="pw"
)

# Prepare some data to be migrated

## Prebuilt custom workerpool

In [ ]:
try:
    # syft absolute
    from syft import test_settings

    external_registry = test_settings.get("external_registry", default="docker.io")
except Exception:
    # Older syft versions do not have test_settings
    external_registry = "docker.io"

external_registry

In [ ]:
worker_docker_tag = f"openmined/bigquery:{sy.__version__}"
worker_docker_tag

In [ ]:
docker_config = sy.PrebuiltWorkerConfig(tag=f"{external_registry}/{worker_docker_tag}")
docker_config

In [ ]:
result = client.api.services.worker_image.submit(worker_config=docker_config)
result

In [ ]:
worker_image = client.images.get_all()[1]
worker_image

In [ ]:
result = client.api.services.image_registry.add(external_registry)
result

In [ ]:
worker_pool_name = "bigquery-pool"

In [ ]:
result = client.api.services.worker_pool.launch(
    pool_name=worker_pool_name,
    image_uid=worker_image.id,
    num_workers=1,
)
result

## Dataset, Code, Job

In [ ]:
client_ds = server.login(email="ds@openmined.org", password="pw")

In [ ]:
dataset = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=np.array([10, 11, 12, 13, 14]),
            data=np.array([[15, 16, 17, 18, 19] for _ in range(100_000)]),
            mock_is_real=True,
        )
    ],
)

client.upload_dataset(dataset)

In [ ]:
data_low = client_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(datasite, data) -> float:
    # launch another job
    print("Computing mean...")
    return data

In [ ]:
req = client_ds.code.request_code_execution(compute_mean)

In [ ]:
client.requests[0].approve()

In [ ]:
job = client_ds.code.compute_mean(data=data_low, blocking=False)

In [ ]:
res = job.wait()

In [ ]:
assert isinstance(res.get(), np.ndarray)

In [ ]:
# todo: add more data

In [ ]:
if server.server_type.value == "python":
    server.land()